In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from scipy import stats
from tableone import TableOne
wd = '/Users/timvigers/Dropbox/Work/Kim Driscoll/BBGD/'
# import warnings;
# warnings.filterwarnings('ignore');

In [ ]:
# Enable cell magic for Rpy2 interface
%load_ext rpy2.ipython

# Participant and Data Characteristics

## Table 1: Participant Characteristics at Visit 1

In [ ]:
df = pd.read_csv(wd+'Data_Clean/bbgd_master.csv')
# Make pretty variables for Table 1
df["Private Insurance"] = ['Yes' if i == 1 else 'No' if i==0 else np.nan for i in df['p_insurance_commercial']]
df["Total Household Income"] = df["par_income"].replace({1:"Under $5K",2:"$5K-$9,999",3:"$10K-$19,999",4:"$20K-29,999", 
                                                   5:"$30K-39,999",6:"$40K-49,999",7:"$50K-59,999",8:"$60K-69,999",
                                                   9:"$70K-79,999",10:"$80K-89,999",11:"$90K-99,999",12:"$100K +"})
df["Parent's Marital Status"] = df["par_marital"].replace({1:"Married to Child's Biologic Father",2:"Married to Child's Stepfather", 
                                                  3:"Separated",4:"Divorced",5:"Widowed",6:"Single",7:"Living with Domestic Partner"})
df["Highest Degree Parent Completed"] = df["par_degree"].replace({1:"No Degree",2:"High School",3:"Associate's",4:"Bachelor's",
                                                                  5:"Master's",6:"Doctoral"})
df["Parent Ethnicity"] = df["par_ethnicity"].replace({1:"Hispanic or Latino",2:"Not Hispanic or Latino"})
df["Parent Race"] = df["par_race"].replace({1:"American Indian or Alaskan Native",2:"Asian",3:"Black or African American",
                                            4:"Native Hawaiian or Pacific Islander",5:"White"})
df["Primary Male Caretaker"] = df["par_race"].replace({1:"Father",2:"Stepfather",3:"Grandfather",4:"Uncle",5:"Mother's boyfriend/partner",6:"Other"})
# Main outcomes
df["Parent Maintain High BG of HFS"] = df["p_maintain_high"]
df["Parent Worry/Helplessness Subscale of HFS"] = df["p_helpless"]
df["Parent Social Consequences Subscale of HFS"] = df["p_social"]
df["Parent GAD-7 Score"] = df["p_gad_total"]
# Make table 1
cols = ["Parent Maintain High BG of HFS","Parent Worry/Helplessness Subscale of HFS",
        "Parent Social Consequences Subscale of HFS","Parent GAD-7 Score",
        "Private Insurance","Total Household Income","Parent's Marital Status",
       "Highest Degree Parent Completed","Parent Race","Parent Ethnicity","Primary Male Caretaker"]

# Print
t1 = TableOne(df.query("studyvisit == 1"),columns=cols,groupby='treatment_group',pval=True,display_all=True,
             normal_test=True,tukey_test=True,dip_test=True)
t1

Highest Degree Parent Completed, Total Household Income, and Parent's Marital Status were not significant by Fisher's exact test either (p = 0.063, 0.969, and 0.297 respectively).

## Outcome Plots

Two of the four outcomes were reasonably normally distributed, but 'p_social' and 'p_gad_total' were skewed and unable to be log-transformed due to many 0 values.

In [ ]:
# Define main outcomes for plotting
var_list = ["Parent Maintain High BG of HFS","Parent Worry/Helplessness Subscale of HFS","Parent Social Consequences Subscale of HFS","Parent GAD-7 Score"]
# Plot
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
df[var_list].hist(ax = ax);# Semicolon hides the matplotlib descriptions in Jupyter

# Model Results
All four outcomes were evaluated using a linear mixed model with random intercept for participant. Model assumptions were checked visually. Study visit was treated as a categorical variable to help account for non-linear trends and varying time to visits. In the following tables, "C(studyvisit)[T.2]" refers to study visit 2, "C(studyvisit)[T.3]" to visit 3, etc.

Income was split into three groups: Under \\$5K-\\$29,999 (reference group); \\$30K-\\$69,999 ("C(par_income)[T.Interval(4, 8, closed='right')]"); and \\$70K-\\$100K+ ("C(par_income)[T.Interval(8, 12, closed='right')]").

In [ ]:
# Import age information
v1_dates = pd.read_csv(wd+'Data_Clean/v1_dates.csv',usecols=['ID','T1','T1D_Onset'])
v1_dates.rename(columns={'ID':'participant_id','T1':'t1_date','T1D_Onset':'t1d_onset'},inplace=True)
# Merge, convert to date
df = df.merge(v1_dates,how='left',on='participant_id')
df['t1_date']=pd.to_datetime(df['t1_date'], errors='coerce', format='%m/%d/%y')
df['t1d_onset']=pd.to_datetime(df['t1d_onset'], errors='coerce', format='%m/%d/%y')
# Calculate age at T1
df['c_dob']= pd.to_datetime(df['c_dob'], errors='coerce', format='%m/%d/%y')
df['age']=df['t1_date']-df['c_dob']
df['age']=[float(t.days)/365.25 for t in df['age']]
# Calculate T1D duration at T1
df['t1d_duration']=df['t1_date']-df['t1d_onset']
df['t1d_duration']=[float(t.days)/365.25 for t in df['t1d_duration']]
# Cut income into three levels, fill down 
df['par_income'] = df.groupby('participant_id')['par_income'].ffill()
df['par_income'] = pd.cut(df['par_income'],[0,4,8,12])

In [ ]:
# Residual plotting function
def plot_resid(model):
    fig, ax = plt.subplots(1,2,figsize = (15,10))
    res = model.resid
    kde = sns.kdeplot(mdf.resid, fill = True,ax=ax[0])
    kde.set_title("KDE Plot of Model Residuals")
    kde.set_xlabel("Residuals")
    qq = sm.qqplot(res,line='s',ax=ax[1])

## Raw Scores

### Parent Maintain High BG of HFS

#### Adjusted for Age

In [ ]:
# Fit a random intercept for each participant
ri_mod = smf.mixedlm("p_maintain_high ~ C(studyvisit)*C(treatment_group)+par_income+age", 
                     df, groups=df["participant_id"],missing = "drop")
mdf = ri_mod.fit()
results_summary = mdf.summary()
results_summary.tables[1]

In [ ]:
plot_resid(mdf)

#### Adjusted for T1D Duration

In [ ]:
# Fit a random intercept for each participant
ri_mod = smf.mixedlm("p_maintain_high ~ C(studyvisit)*C(treatment_group)+par_income+t1d_duration", 
                     df, groups=df["participant_id"],missing = "drop")
mdf = ri_mod.fit()
results_summary = mdf.summary()
results_summary.tables[1]

In [ ]:
plot_resid(mdf)

### Parent Worry/Helplessness Subscale of HFS

#### Adjusted for Age

In [ ]:
# Fit a random intercept for each participant
ri_mod = smf.mixedlm("p_helpless ~ C(studyvisit)*C(treatment_group)+par_income+age", 
                     df, groups=df["participant_id"],missing = "drop")
mdf = ri_mod.fit()
results_summary = mdf.summary()
results_summary.tables[1]

In [ ]:
plot_resid(mdf)

#### Adjusted for T1D Duration

In [ ]:
# Fit a random intercept for each participant
ri_mod = smf.mixedlm("p_helpless ~ C(studyvisit)*C(treatment_group)+par_income+t1d_duration", 
                     df, groups=df["participant_id"],missing = "drop")
mdf = ri_mod.fit()
results_summary = mdf.summary()
results_summary.tables[1]

In [ ]:
plot_resid(mdf)

### Parent Social Consequences Subscale of HFS

#### Adjusted for Age

In [ ]:
# Fit a random intercept for each participant
ri_mod = smf.mixedlm("p_social ~ C(studyvisit)*C(treatment_group)+par_income+age", 
                     df, groups=df["participant_id"],missing = "drop")
mdf = ri_mod.fit()
results_summary = mdf.summary()
results_summary.tables[1]

In [ ]:
plot_resid(mdf)

#### Adjusted for T1D Duration

In [ ]:
# Fit a random intercept for each participant
ri_mod = smf.mixedlm("p_social ~ C(studyvisit)*C(treatment_group)+par_income+t1d_duration", 
                     df, groups=df["participant_id"],missing = "drop")
mdf = ri_mod.fit()
results_summary = mdf.summary()
results_summary.tables[1]

In [ ]:
plot_resid(mdf)

### Parent GAD-7 Score

#### Adjusted for Age

In [ ]:
# Fit a random intercept for each participant
ri_mod = smf.mixedlm("p_gad_total ~ C(studyvisit)*C(treatment_group)+par_income+age", 
                     df, groups=df["participant_id"],missing = "drop")
mdf = ri_mod.fit()
results_summary = mdf.summary()
results_summary.tables[1]

In [ ]:
plot_resid(mdf)

#### Adjusted for T1D Duration

In [ ]:
# Fit a random intercept for each participant
ri_mod = smf.mixedlm("p_gad_total ~ C(studyvisit)*C(treatment_group)+par_income+t1d_duration", 
                     df, groups=df["participant_id"],missing = "drop")
mdf = ri_mod.fit()
results_summary = mdf.summary()
results_summary.tables[1]

In [ ]:
plot_resid(mdf)

## Elevated Scores vs. Not Elevated

All four outcomes were evaluated using a logistic mixed model with random intercept for participant. Coefficient estimates are reported on the log scale.

In [30]:
# Make cutoffs:
# GAD-7 >=10 considered elevated
# Maintain High BG >=7 considered elevated
# Helplessness >=24 considered elevated
# Social Consequences >=9 considered elevated
df['p_elevated_maintain']=[float(1) if m >= 7 else float(0) if m < 7 else np.nan for m in df['p_maintain_high']]
df['p_elevated_helpless']=[float(1) if m >= 24 else float(0) if m < 24 else np.nan for m in df['p_helpless']]
df['p_elevated_social']=[float(1) if m >= 9 else float(0) if m < 9 else np.nan for m in df['p_social']]
df['p_elevated_gad']=[float(1) if m >= 10 else float(0) if m < 10 else np.nan for m in df['p_gad_total']]
# Make simplified DF for R
log_df = df[['participant_id','studyvisit','treatment_group','par_income','age','t1d_duration',
             'p_elevated_maintain','p_elevated_helpless','p_elevated_social','p_elevated_gad']].copy().dropna()
log_df['par_income'] = log_df['par_income'].astype('str')

### Parent Maintain High BG of HFS

#### Adjusted for Age

In [ ]:
%%R -i log_df
suppressMessages(library(lme4))
suppressMessages(library(lmerTest))
mod <- glmer(p_elevated_maintain ~ factor(studyvisit)*factor(treatment_group)+factor(par_income)+age+(1|participant_id),data = log_df,family = binomial(link = "logit"))
print(summary(mod)$coefficients[,c(1,4)])

#### Adjusted for T1D Duration

In [ ]:
%%R -i log_df
suppressMessages(library(lme4))
suppressMessages(library(lmerTest))
mod <- glmer(p_elevated_maintain ~ factor(studyvisit)*factor(treatment_group)+factor(par_income)+t1d_duration+(1|participant_id),data = log_df,family = binomial(link = "logit"))
print(summary(mod)$coefficients[,c(1,4)])

### Parent Worry/Helplessness Subscale of HFS

#### Adjusted for Age

In [31]:
%%R -i log_df
suppressMessages(library(lme4))
suppressMessages(library(lmerTest))
mod <- glmer(p_elevated_helpless ~ factor(studyvisit)*factor(treatment_group)+factor(par_income)+age+(1|participant_id),data = log_df,family = binomial(link = "logit"))
print(summary(mod)$coefficients[,c(1,4)])

                                                 Estimate   Pr(>|z|)
(Intercept)                                  -4.354672606 0.19742613
factor(studyvisit)2                           1.265994906 0.29154967
factor(studyvisit)3                          -3.430425887 0.03312979
factor(studyvisit)4                          -1.263320729 0.32753177
factor(studyvisit)5                           0.006608137 0.99597058
factor(studyvisit)6                          -0.595844491 0.63598443
factor(studyvisit)7                          -0.355464173 0.77304081
factor(studyvisit)8                           0.227307702 0.84478853
factor(treatment_group)1                      3.309591063 0.04095204
factor(par_income)(4, 8]                      1.794736700 0.46671360
factor(par_income)(8, 12]                     1.852554680 0.40430626
age                                          -0.041226903 0.81503457
factor(studyvisit)2:factor(treatment_group)1 -2.421744536 0.13155422
factor(studyvisit)3:factor(treatme

#### Adjusted for T1D Duration

In [32]:
%%R -i log_df
suppressMessages(library(lme4))
suppressMessages(library(lmerTest))
mod <- glmer(p_elevated_helpless ~ factor(studyvisit)*factor(treatment_group)+factor(par_income)+t1d_duration+(1|participant_id),data = log_df,family = binomial(link = "logit"))
print(summary(mod)$coefficients[,c(1,4)])

                                                Estimate   Pr(>|z|)
(Intercept)                                  -6.58490061 0.04104814
factor(studyvisit)2                           1.11002248 0.35165510
factor(studyvisit)3                          -3.73983745 0.02241959
factor(studyvisit)4                          -1.54043953 0.23770607
factor(studyvisit)5                          -0.18878223 0.88508708
factor(studyvisit)6                          -0.84218043 0.50571346
factor(studyvisit)7                          -0.54165013 0.65805368
factor(studyvisit)8                           0.04343862 0.97001680
factor(treatment_group)1                      3.11568102 0.05576728
factor(par_income)(4, 8]                      2.56189840 0.34242920
factor(par_income)(8, 12]                     2.39140427 0.31649018
t1d_duration                                  0.19225151 0.29423227
factor(studyvisit)2:factor(treatment_group)1 -2.33936669 0.14680270
factor(studyvisit)3:factor(treatment_group)1 -1.

### Parent Social Consequences Subscale of HFS

#### Adjusted for Age

In [33]:
%%R -i log_df
suppressMessages(library(lme4))
suppressMessages(library(lmerTest))
mod <- glmer(p_elevated_social ~ factor(studyvisit)*factor(treatment_group)+factor(par_income)+age+(1|participant_id),data = log_df,family = binomial(link = "logit"))
print(summary(mod)$coefficients[,c(1,4)])

                                                Estimate    Pr(>|z|)
(Intercept)                                   -4.9624551 0.284389585
factor(studyvisit)2                            0.8884280 0.844436853
factor(studyvisit)3                            7.8696466 0.009980582
factor(studyvisit)4                           -0.3845831 0.906177309
factor(studyvisit)5                            0.5858532 0.890760220
factor(studyvisit)6                          -14.6650311 0.997505362
factor(studyvisit)7                            0.3819080 0.925578176
factor(studyvisit)8                            2.4507189 0.411244455
factor(treatment_group)1                       5.0901791 0.114428109
factor(par_income)(4, 8]                       2.4646172 0.479410902
factor(par_income)(8, 12]                      0.2200248 0.946688755
age                                           -0.2734210 0.158419416
factor(studyvisit)2:factor(treatment_group)1  -2.0587172 0.661920569
factor(studyvisit)3:factor(treatme

#### Adjusted for T1D Duration

In [34]:
%%R -i log_df
suppressMessages(library(lme4))
suppressMessages(library(lmerTest))
mod <- glmer(p_elevated_social ~ factor(studyvisit)*factor(treatment_group)+factor(par_income)+t1d_duration+(1|participant_id),data = log_df,family = binomial(link = "logit"))
print(summary(mod)$coefficients[,c(1,4)])

                                                Estimate    Pr(>|z|)
(Intercept)                                   -6.7490755 0.104911519
factor(studyvisit)2                            0.6534093 0.843947733
factor(studyvisit)3                            7.9173105 0.007295597
factor(studyvisit)4                           -0.4195572 0.878233238
factor(studyvisit)5                            0.4209178 0.895605213
factor(studyvisit)6                          -15.9903997 0.980302482
factor(studyvisit)7                            0.4483691 0.883831090
factor(studyvisit)8                            2.3007158 0.350074536
factor(treatment_group)1                       4.5513054 0.100585097
factor(par_income)(4, 8]                       1.8722273 0.555905897
factor(par_income)(8, 12]                     -0.3736559 0.897805932
t1d_duration                                  -0.1475118 0.478683490
factor(studyvisit)2:factor(treatment_group)1  -1.7591474 0.623766037
factor(studyvisit)3:factor(treatme

### Parent GAD-7 Score

#### Adjusted for Age

In [35]:
%%R -i log_df
suppressMessages(library(lme4))
suppressMessages(library(lmerTest))
mod <- glmer(p_elevated_gad ~ factor(studyvisit)*factor(treatment_group)+factor(par_income)+age+(1|participant_id),data = log_df,family = binomial(link = "logit"))
print(summary(mod)$coefficients[,c(1,4)])

                                                Estimate   Pr(>|z|)
(Intercept)                                  -0.42980278 0.79941059
factor(studyvisit)2                           1.07166001 0.24432866
factor(studyvisit)3                          -0.15202846 0.86264653
factor(studyvisit)4                           1.13593940 0.21048892
factor(studyvisit)5                          -0.87805051 0.33156341
factor(studyvisit)6                           0.87872370 0.33204019
factor(studyvisit)7                           0.14121511 0.86932063
factor(studyvisit)8                          -0.87550480 0.29133574
factor(treatment_group)1                      0.85252121 0.42000396
factor(par_income)(4, 8]                      3.02512891 0.01629773
factor(par_income)(8, 12]                     1.99908921 0.06691951
age                                          -0.03945028 0.67717497
factor(studyvisit)2:factor(treatment_group)1 -0.59650425 0.67917682
factor(studyvisit)3:factor(treatment_group)1 -0.

#### Adjusted for T1D Duration

In [36]:
%%R -i log_df
suppressMessages(library(lme4))
suppressMessages(library(lmerTest))
mod <- glmer(p_elevated_gad ~ factor(studyvisit)*factor(treatment_group)+factor(par_income)+t1d_duration+(1|participant_id),data = log_df,family = binomial(link = "logit"))
print(summary(mod)$coefficients[,c(1,4)])

                                                Estimate   Pr(>|z|)
(Intercept)                                   0.04494058 0.97583765
factor(studyvisit)2                           1.06776400 0.24544090
factor(studyvisit)3                          -0.17022415 0.84631058
factor(studyvisit)4                           1.12507777 0.21389856
factor(studyvisit)5                          -0.87897326 0.32925924
factor(studyvisit)6                           0.87572056 0.33304743
factor(studyvisit)7                           0.14767785 0.86288888
factor(studyvisit)8                          -0.87986608 0.28792427
factor(treatment_group)1                      0.84668518 0.41870127
factor(par_income)(4, 8]                      2.62263369 0.04357088
factor(par_income)(8, 12]                     1.70303200 0.12692512
t1d_duration                                 -0.10015170 0.29356615
factor(studyvisit)2:factor(treatment_group)1 -0.63071986 0.66203876
factor(studyvisit)3:factor(treatment_group)1 -0.